In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install  streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 122.4 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.0
    Uninstalling cachetools-7.0.0:
      Successfully uninstalled cachetools-7.0.0


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import requests
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

# --- CONFIGURATION (Must be the first Streamlit command) ---
st.set_page_config(
    page_title="Crypto Risk Commander",
    page_icon="⚡",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- SESSION STATE INITIALIZATION ---
if "logged_in" not in st.session_state:
    st.session_state["logged_in"] = False
if "username" not in st.session_state:
    st.session_state["username"] = None
# A simple mock database for users.
if "users" not in st.session_state:
    st.session_state["users"] = {"admin": "1234"}
if "page" not in st.session_state:
    st.session_state["page"] = "login"

# --- CUSTOM CSS FOR MODERN LOOK ---
st.markdown("""
    <style>
    .main {
        background-color: #0e1117;
    }
    .metric-card {
        background-color: #262730;
        border: 1px solid #41424b;
        padding: 15px;
        border-radius: 10px;
        color: white;
    }
    h1, h2, h3 {
        color: #fafafa;
    }
    div.stButton > button {
        width: 100%;
        background-color: #ff4b4b;
        color: white;
        font-weight: bold;
    }
    /* Style the login container */
    .login-container {
        padding: 2rem;
        background-color: #262730;
        border-radius: 10px;
        box-shadow: 0 4px 6px rgba(0,0,0,0.3);
    }
    </style>
    """, unsafe_allow_html=True)

# --- AUTHENTICATION FUNCTIONS ---
def show_login():
    st.title("🔐 Login to Crypto Risk Commander")
    st.markdown("Please sign in to access professional risk analysis tools.")

    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        with st.form("login_form"):
            username = st.text_input("Username", placeholder="admin")
            password = st.text_input("Password", type="password", placeholder="1234")

            submit = st.form_submit_button("Sign In")

            if submit:
                if username in st.session_state["users"] and st.session_state["users"][username] == password:
                    st.session_state["logged_in"] = True
                    st.session_state["username"] = username
                    st.success("Login Successful!")
                    st.rerun()
                else:
                    st.error("Invalid Username or Password")

        st.markdown("---")
        if st.button("Create New Account"):
            st.session_state["page"] = "register"
            st.rerun()

        st.info("💡 **Demo Creds:** Username: `admin`, Password: `1234`")

def show_register():
    st.title("📝 Create Account")

    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        with st.form("register_form"):
            new_user = st.text_input("Choose Username")
            new_pass = st.text_input("Choose Password", type="password")
            confirm_pass = st.text_input("Confirm Password", type="password")

            submit = st.form_submit_button("Register")

            if submit:
                if new_user in st.session_state["users"]:
                    st.error("Username already exists!")
                elif new_pass != confirm_pass:
                    st.error("Passwords do not match!")
                elif not new_user or not new_pass:
                    st.error("Please fill out all fields.")
                else:
                    st.session_state["users"][new_user] = new_pass
                    st.success("Account Created! Please Login.")
                    st.session_state["page"] = "login"
                    st.rerun()

        if st.button("Back to Login"):
            st.session_state["page"] = "login"
            st.rerun()

# --- DATA & ANALYSIS FUNCTIONS ---
@st.cache_data(ttl=600)  # Cache for 10 minutes
def fetch_market_data(coin_id, days):
    try:
        url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
        params = {"vs_currency": "usd", "days": days}
        response = requests.get(url, params=params)
        data = response.json()
        if "prices" not in data: return None
        prices = data["prices"]
        df = pd.DataFrame(prices, columns=["timestamp", "price"])
        df["date"] = pd.to_datetime(df["timestamp"], unit="ms")
        df.set_index("date", inplace=True)
        return df
    except Exception: return None

# --- DASHBOARD LOGIC ---
def show_dashboard():
    # Sidebar - User Info & Logout
    st.sidebar.title(f"👤 {st.session_state['username']}")
    if st.sidebar.button("Logout"):
        st.session_state["logged_in"] = False
        st.session_state["username"] = None
        st.rerun()

    # Sidebar - Settings
    st.sidebar.header("🔍 Analysis Settings")
    available_coins = {
        "Bitcoin": "bitcoin",
        "Ethereum": "ethereum",
        "Solana": "solana",
        "Dogecoin": "dogecoin",
        "Cardano": "cardano",
        "Ripple": "ripple",
    }
    selected_coin_name = st.sidebar.selectbox("Select Cryptocurrency", list(available_coins.keys()))
    selected_coin_id = available_coins[selected_coin_name]

    timeframe_map = {"Last 30 Days": "30", "Last 90 Days": "90", "Last Year": "365"}
    selected_timeframe = st.sidebar.selectbox("Select Time Horizon", list(timeframe_map.keys()))
    days = timeframe_map[selected_timeframe]

    # Sidebar - Manual Simulator
    st.sidebar.markdown("---")
    st.sidebar.subheader("🛠️ Manual Scenario Simulator")
    manual_price_change = st.sidebar.number_input(
        "Enter Daily Price Change (%)", min_value=0.0, max_value=100.0, value=5.0, step=0.5
    )
    analyze_button = st.sidebar.button("💥 Analyze Risk Scenario")

    # --- MAIN CONTENT ---
    st.title(f"⚡ {selected_coin_name} Risk Analysis Dashboard")

    # 1. Manual Analysis Result
    if analyze_button:
        st.markdown("### 🧪 Manual Scenario Results")
        if manual_price_change > 15:
            m_risk, m_color = "EXTREME RISK ☢️", "#ff2b2b"
            m_msg = "Catastrophic! Indicates a crash or major manipulation."
        elif manual_price_change > 7:
            m_risk, m_color = "HIGH RISK 🔥", "#ff9f1c"
            m_msg = "Significant movement. Typical for altcoins or news events."
        elif manual_price_change > 3:
            m_risk, m_color = "MODERATE RISK ⚖️", "#fbd400"
            m_msg = "Standard daily volatility for crypto."
        else:
            m_risk, m_color = "LOW RISK 🛡️", "#2ec4b6"
            m_msg = "Very stable market conditions."

        st.markdown(f"""
        <div style="padding: 20px; border-radius: 10px; background-color: {m_color}22; border-left: 5px solid {m_color};">
            <h3 style="margin:0; color: {m_color};">{m_risk}</h3>
            <p style="margin-top: 10px; font-size: 1.1em;">{m_msg}</p>
        </div>
        """, unsafe_allow_html=True)
        st.divider()

    # 2. Real-time Analysis
    st.markdown(f"### 📊 Live Market Data: {selected_coin_name}")
    with st.spinner(f"Connecting to live market data for {selected_coin_name}..."):
        df = fetch_market_data(selected_coin_id, days)

    if df is not None:
        # Metrics Calculation
        df['daily_return'] = df['price'].pct_change() * 100
        current_price = df['price'].iloc[-1]
        vol = df['daily_return'].std() * np.sqrt(365)
        max_drawdown = ((df['price'] - df['price'].cummax()) / df['price'].cummax() * 100).min()

        if vol > 100: r_color, r_level = "#ff2b2b", "EXTREME ☢️"
        elif vol > 60: r_color, r_level = "#ff9f1c", "HIGH 🔥"
        elif vol > 30: r_color, r_level = "#fbd400", "MODERATE ⚖️"
        else: r_color, r_level = "#2ec4b6", "LOW 🛡️"

        # KPIs
        c1, c2, c3, c4 = st.columns(4)
        c1.metric("Current Price", f"${current_price:,.2f}")
        c2.metric("Annualized Volatility", f"{vol:.1f}%")
        c3.metric("Max Drawdown", f"{max_drawdown:.1f}%")
        c4.markdown(f"""
            <div style="text-align: center; padding: 5px; border-radius: 5px; background-color: {r_color}33; border: 1px solid {r_color};">
                <h4 style="margin:0; color: {r_color};">{r_level}</h4>
            </div>
            """, unsafe_allow_html=True)

        # Charts
        processed_df = df.copy()
        processed_df['7D_MA'] = processed_df['price'].rolling(window=7).mean()

        st.subheader("Price Trend")
        fig_price = go.Figure()
        fig_price.add_trace(go.Scatter(x=processed_df.index, y=processed_df['price'], mode='lines', name='Price', line=dict(color='#00d4ff')))
        fig_price.add_trace(go.Scatter(x=processed_df.index, y=processed_df['7D_MA'], mode='lines', name='7-Day MA', line=dict(color='#ffab00', dash='dash')))
        fig_price.update_layout(template="plotly_dark", paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', height=400)
        st.plotly_chart(fig_price, use_container_width=True)

        c_chart1, c_chart2 = st.columns(2)
        with c_chart1:
            st.caption("Daily Returns Distribution")
            fig_hist = px.histogram(processed_df, x="daily_return", nbins=40, color_discrete_sequence=['#7b2cbf'])
            fig_hist.update_layout(template="plotly_dark", paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
            st.plotly_chart(fig_hist, use_container_width=True)
        with c_chart2:
            st.caption("Rolling Volatility (Risk Intensity)")
            processed_df['rolling_vol'] = processed_df['daily_return'].rolling(window=14).std()
            fig_vol = px.area(processed_df, x=processed_df.index, y='rolling_vol', color_discrete_sequence=['#ff0054'])
            fig_vol.update_layout(template="plotly_dark", paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
            st.plotly_chart(fig_vol, use_container_width=True)

    else:
        st.error("⚠️ API Error: Used too many requests. Please wait 1 minute.")

# --- APP FLOW CONTROLLER ---
if st.session_state["logged_in"]:
    show_dashboard()
else:
    if st.session_state["page"] == "register":
        show_register()
    else:
        show_login()

Overwriting app.py


In [ ]:
import requests
import pandas as pd
import numpy as np
import time # Import time module for delays

# List of cryptocurrencies
coins = ["bitcoin", "ethereum", "dogecoin"]

results = []

for coin in coins:
    print(f"Fetching data for {coin}...")

    # API URL (CoinGecko)
    url = f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart"
    params = {
        "vs_currency": "usd",
        "days": "30"   # last 30 days
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status() # Raise an exception for HTTP errors (4xx or 5xx)
        data = response.json()

        if "prices" not in data:
            print(f"Error: 'prices' key not found in data for {coin}. Response: {data}")
            continue

        prices = data["prices"]

        # Create DataFrame
        df = pd.DataFrame(prices, columns=["timestamp", "price"])

        # Convert timestamp to date
        df["date"] = pd.to_datetime(df["timestamp"], unit="ms")
        df = df[["date", "price"]]

        # Save daily data
        df.to_csv(f"{coin}_daily_prices.csv", index=False)

        # Volatility calculation
        volatility = np.std(df["price"])

        # Risk classification
        if volatility < 500:
            risk = "Low Risk"
        elif volatility < 2000:
            risk = "Medium Risk"
        else:
            risk = "High Risk"

        results.append([coin.upper(), round(volatility, 2), risk])

    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error for {coin}: {errh}")
        print(f"Response status code: {response.status_code}")
        print(f"Response content: {response.text}")
        # If it's a 429 Too Many Requests, implement a delay
        if response.status_code == 429:
            print("Rate limit hit. Waiting for 60 seconds before retrying...")
            time.sleep(60) # Wait for a minute
    except requests.exceptions.RequestException as err:
        print(f"Error connecting to CoinGecko API for {coin}: {err}")
    except ValueError as e: # Catches JSONDecodeError directly
        print(f"JSON decoding error for {coin}: {e}")
        print(f"Response content that caused error: {response.text}")
    except Exception as e:
        print(f"An unexpected error occurred for {coin}: {e}")

# Final result table
result_df = pd.DataFrame(results, columns=["Crypto", "Volatility", "Risk Level"])
print("\nCrypto Volatility Summary:")
print(result_df)

# Save summary
result_df.to_csv("crypto_volatility_risk_summary.csv", index=False)


Fetching data for bitcoin...
Fetching data for ethereum...
Fetching data for dogecoin...
HTTP Error for dogecoin: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/dogecoin/market_chart?vs_currency=usd&days=30
Response status code: 429
Response content: {"status":{"error_code":429,"error_message":"You've exceeded the Rate Limit. Please visit https://www.coingecko.com/en/api/pricing to subscribe to our API plans for higher rate limits."}}
Rate limit hit. Waiting for 60 seconds before retrying...

Crypto Volatility Summary:
     Crypto  Volatility Risk Level
0   BITCOIN     8038.57  High Risk
1  ETHEREUM      407.82   Low Risk


In [ ]:
!pip install pyngrok -q

In [ ]:
from pyngrok import ngrok

# Replace 'YOUR_AUTHTOKEN' with your actual ngrok authentication token
# You can find your authtoken at https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token('39A8OmERwQezjQAUJ77HsMoVF2C_3AJzJQ15zNDmCv4rtDqnE')

In [ ]:
from pyngrok import ngrok
import os
import time

# Kill any previously running streamlit processes to ensure the latest app.py is loaded
os.system("killall streamlit")

# Start streamlit in the background
os.system("streamlit run app.py &")

# Give Streamlit a moment to start
time.sleep(5)

# Create public URL
public_url = ngrok.connect(8501)
print(public_url)


NgrokTunnel: "https://jakobe-unhospitalized-bo.ngrok-free.dev" -> "http://localhost:8501"
